<a href="https://colab.research.google.com/github/projectwint/practice_AI/blob/main/keras_%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%B6%84%EB%A5%982.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이진 분류 코드(binary classification)

이미지와 카테고리를 X, Y 값으로 나누기
</br>
</br>
최종적으로 image_data.npy의 형태로 저장된다.
</br>
출처: https://github.com/lsjsj92/keras_basic/blob/master/6.%20predict_binary_img_with_CNN.ipynb
<br>
https://lsjsj92.tistory.com/357

In [ ]:
from PIL import Image
import os, glob, sys, numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

img_dir = './binary_data/imgs' #이미지 경로
categories = ['ground', 'water']
np_classes = len(categories)

#이미지 크기: 128x128 px
image_w = 128
image_h = 128

pixel = image_h*image_w*3

X = []
y = []

for idx, cat in enumerate(categories):
  img_dir_detail = img_dir + "/" + cat
  files = glob.glob(img_dir_detail + "/*.jpg")

  for i, f in enumerate(files):
    try:
      img = Image.open(f)
      img = img.convert("RGB")
      img = img.resize((image_w, image_h))
      data = np.asarray(img)
      #Y는 0, 1이기 때문에 idx 값으로 넣는다.
      X.append(data)
      y.append(idx)
      if i % 300 == 0:
        print(cat, " : ", f)
      except:
        print(cat, str(i) + "번째에서 에러")

X = np.array(X)
Y = np.array(y)

X_train, X_test, Y_train, Y_test, Y_test = train_test_split(X, Y, test_size = 0.1)
xy = (X_train, X_test, Y_train, Y_test)
np.save("./numpy_data/binary_image_data.npy", xy)

출처: https://www.daleseo.com/python-enumerate/

enumerate() 함수
기본적으로 인덱스와 원소로 이루어진 tuple을 만들어준다. 
인덱스와 원소를 다른 변수에 할당하고 싶으면 unpacking을 해야 한다. 
</br></br>
아래의 코드에서는 idx에 index를, cat에 원소가 할당되는 것. 
</br>
</br>
+print(idx, cat) </br>
0 ground </br>
1 water

+print(label, cat)</br>
[1, 0] ground </br>
[0, 1] water

CNN 컨볼루션 신경망 모델, Conv2D로 만들 수 있다.
</br>
방금 만든 npy 파일을 이용해서 훈련한다.
</br></br>
컨볼루션이나 맥스룰링은 2차원인데 전결합층에 전달하기 위해서는 1차원으로 바꿔줘야 한다. 
이 때 사용하는 것이 **Flatten()** 이다. 

In [ ]:
from keras.models imort Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backen.tensorflow_backend as K

X_train, X_test, y_test, y_test = np.load('./numpy_data/binary_image_data.npy')
print(X_train.shape)
print(X_train.shape[0])
print(np.bincount(y_train))
print(np.bincount(y_test))

In [ ]:
image_w = 128
image_h = 18
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

with K.tf_ops.device('/device:GPU:0'):
  model = Sequential()
  model.add(Con2D(32, (3, 3), padding = "same", input_shape = X_train.shpae[1:], activation = "relu"))
  model.add(Maxpooling2D(pool_size = (2, 2)))
  #Conv2D를 이용
  #3x3 크기의 컨볼루션 레이어를 32개의 필터수만큼 처음에 생성한다.
  #활성화 함수는 relu를 사용

  model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Dropout(0.25))
  #입력형태(input_shpae)는 128 x 128이기 때문에 (128, 128, 3)의 tuple 값을 가짐
  #MaxPooling2D: 주요값을 뽑아내서 작은 출력값을 만들고, 사소한 변화는 무시한다. 

  model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  #CNN에서 컨볼루션 레이어를 거치면 주요 특징만 추출돼서 학습됨. 
  model.add(Dense(256, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation = "sigmoid"))

  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  #2진분류(카테고리가 2개)기 때문에 binary 값으로 출력을 지정한다. 

  model_dir = './model'

  if not os.path.exists(model_dir):
    os.mkdir(model_dir)
  model_path = model_dir + "/ground_water_classify.model"

  checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbos = 1, save_best_only = True)
  early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5)
  #val_loss 값이 5번 이상 상승되지 않으면 자동으로 멈추는 코드(overfitting 방지)

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 100, validation_split = 0.15, callbacks = [checkpoint, early_stopping])

In [ ]:
print("정확도 : %.2f" %(model.evaluate(X_test, y_test)[1]))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(histroy.history['val_loss'])
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss', 'acc', 'val_acc'], loc = 'upper left')

plt. show()

In [ ]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

import tensorflow as tf

seed = 5
tf.set_random_seed(seed)
np.random.seed(seed)

caltech_dir = './binary_img_data/img_test'

image_w = 128
image_h = 128

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir + "/*/*.*")

for i , f in enumerate(files):
  img = Image.open(f)
  img = img.convert("RGB")
  img = img.resize((image_w, image_h))
  data = np.asarray(img)

  filenames.append(f)
  X.append(data)

X = np.array(X)
X = X.astype(float) / 255
model = load_model('./model/ground_water_classify.model')

prediction = model.predict(X)
np.set_printoption(formatter = {'float' : lambda x : "{0:0.3f}".format(x)})
cnt = 0

for i in prediction:
  if i >= 0.5:
    print("해당" + filenames[cnt].split("\\")[1] + filenames[cnt].split("\\")[2] + " 이미지는 땅으로 추정됩니다.")
  else:
    print("해당" + filenames[cnt].split("\\")[1] + filename[cnt].split("\\")[2] + " 이미지는 물로 추정됩니다.")
  cnt += 1